In [1]:
import pandas as pd
import xlwings as xw
import string
import numpy as np
import time


In [2]:
code = 124404 #Marksans Pharma
other_params = "Millions/INR/5/asc/N2"
typ = "Consolidated"
details_1 = "Abridged"
details_2 = "Actual"
details_3 = ""
details_4 = "Reported"
period = 5
no_of_periods = 5

In [3]:
path = r"D:\BC Personal\Fin model non bank.xlsx"
ABC = list(string.ascii_uppercase)

In [8]:
book = xw.Book()
sheetpl = book.sheets.add("Ace PL")
sheetbs = book.sheets.add("Ace BS")

statement = "Company-Profit And Loss"
Ace_string = "=@EQNXTV(\""+statement+"\","+str(code)+",\""+other_params+"\",\""+typ+"\",\""+details_1+"\",\""\
+details_2+"\")\n" #"\",\""+details_3+"\",\""+details_4+"\")\n"
sheetpl.range("A1").value = Ace_string

statement = "Company-Balance Sheet"
Ace_string_bs = "=@EQNXTV(\""+statement+"\","+str(code)+",\""+other_params+"\",\""+typ+"\",\""+details_1+"\",\""\
+details_2+"\")\n"
sheetbs.range("A1").value = Ace_string_bs

In [9]:

data = sheetpl.range("A10").expand().options(pd.DataFrame, header=0, index=False).value
#print(data)
data.columns=["Head", "FY18", "FY19", "FY20", "FY21", "FY22"]
plheads = pd.read_excel(path, usecols="A:B", sheet_name="Mapping")
df = data.merge(plheads, on="Head")
pl = df.groupby(by="Mapping").sum()#.drop(columns=["Head"])
#print(pl)
#pl.loc["Dividend"]=[0,0,0,0,0]
print(pl)

data = sheetbs.range("A9").expand().options(pd.DataFrame, header=0, index=False).value
print(data)
usr=input()
data.columns=["Head", "FY18", "FY19", "FY20", "FY21", "FY22"]
#print(data)
bsheads = pd.read_excel(path, usecols="A:B", sheet_name="MappingBS")
df = data.merge(bsheads, on="Head")
bs = df.groupby(by="Mapping").sum()#.drop(columns=["Head"])
print(bs)




                               FY18          FY19          FY20      FY21  \
Mapping                                                                     
COGS                    5231.269263   4980.035832   5678.315101   5950.56   
Depreciation             268.382687    228.039029    266.615489    361.51   
Employee expenses       1477.480568   1590.663561   1684.163547   1973.23   
Exceptional expenses       0.000000      0.000000      0.000000      0.00   
Extraordinary items        0.000000      0.000000      0.000000      0.00   
Interest cost            102.202907     96.553918     87.463768     79.86   
Minority interest PL     -28.609929    -39.767483    -37.334094    -25.99   
Other expenses          1142.971186   1424.981755   1735.575746   2106.58   
Other income              84.740182     46.210042      3.277666     66.94   
Sales                   9126.920323  10000.692254  11342.089526  13761.82   
Sales and Mkt expenses   491.372990    683.562347    321.515864    335.38   


                                    FY18         FY19         FY20     FY21  \
Mapping                                                                       
Acc Depreciation             2159.699036  2365.375590  2626.733151  2425.62   
CWIP                            0.000000     0.000000     0.000000   116.80   
Cash and equivalents          398.768569   335.384697   937.270861  2121.83   
Creditors                     778.282714  1060.692017  1103.291395  1686.64   
Debtors                      1769.055770  1765.969384  2433.517408  2719.49   
Equity share capital          409.313698   409.313698   409.313698   409.31   
Gross block                  4953.473916  5062.638178  5653.894623  5519.25   
Inventory                    2181.241268  2932.460805  2418.439338  4043.20   
Long term investments           0.000000     0.000000     0.000000     0.00   
Minority interest BS           95.494128   102.168392   131.101423   188.57   
Other current assets          156.301496   280.6042

In [10]:
for value in plheads["Mapping"].unique():
    if value not in pl.index:
        if pd.isna(value):
            continue
        else:
            pl.loc[value]=[0,0,0,0,0]

for value in bsheads["Mapping"].unique():
    if value not in bs.index:
        if pd.isna(value):
            continue
        else:
            bs.loc[value]=[0,0,0,0,0]


In [11]:
book = xw.Book()
sheetbs = book.sheets.add("BS")
sheetbs.range("A1").value=bs

sheetpl = book.sheets.add("PL")
sheetpl.range("A1").value=pl

shtratios = book.sheets.add("Ratios")
sheetcf = book.sheets.add("CF")

In [12]:
bsheads = bs.index.tolist()
plheads = pl.index.tolist()
ratioheads = []
cfheads = []
formulae = pd.read_excel(path, sheet_name="Formulae", usecols="A:O").to_dict("records")
for i, row in enumerate(formulae):
    variable = row["Head"]
    print("Working on:", variable)

    if row["Sheet"] == "BS":
        #ele1 = "BS!"
        #idx1 = bsheads.index(variable)+2 #+1 as xl starts from 1 and +1 for header row
        sheet = sheetbs
        l = len(bsheads)
    elif row["Sheet"] == "PL":
        #ele1 = "PL!"
        #idx1 = plheads.index(variable)+2
        sheet = sheetpl
        l = len(plheads)
    elif row["Sheet"] == "Ratios":
        #ele1 = "Ratios!"
        #idx1 = ratioheads.index(variable)+2
        sheet = shtratios
        l = len(ratioheads)
    elif row["Sheet"] == "CF":
        sheet = sheetcf
        l = len(cfheads)
    else:
        print("Unknown variable: ", variable)
        continue
    
    headercell = "A"+str(l+2)
    print(sheet, headercell)
    sheet.range(headercell).value = row["Head"]
    #curr_cellref = ele1+ABC[no_of_periods+proj_yr+1]+str(idx1)

    for j in range(len(bs.columns)-1):
        #col = ABC[j+1]
        col = ABC[j+2]
        colprev = ABC[j+1]
        
        keys = list(row.keys())
        formula = "="
        for key in range(1,len(keys)-1): #skipping first and last columns
            if pd.isna(row[keys[key]]):
                continue
            if "Element" in keys[key]:
                if isinstance(row[keys[key]], int) or isinstance(row[keys[key]], float):
                    cellref=str(row[keys[key]])
                else:
                    if "Avg " in row[keys[key]]:
                        element = row[keys[key]].lstrip("Avg ")
                    elif "Delta " in row[keys[key]]:
                        element = row[keys[key]].lstrip("Delta").lstrip(" ")
                        #print("kkk", row[keys[key]], element)
                    else:
                        element = row[keys[key]]
                    if element in bsheads:
                        ele = "BS!"
                        idx = bsheads.index(element)+2 #+1 as xl starts from 1 and +1 for header row
                        sheetref = sheetbs
                    elif element in plheads:
                        ele = "PL!"
                        idx = plheads.index(element)+2
                        sheetref = sheetpl
                    elif element in ratioheads:
                        ele = "Ratios!"
                        idx = ratioheads.index(element)+2
                        sheetref = shtratios
                    elif element in cfheads:
                        ele = "CF!"
                        idx = cfheads.index(element)+2
                        sheetref = sheetcf
                    else:
                        print("Unknown: ", element, type(element))
                        continue

                    if element == variable: #Refers to itself
                        continue#cellref = ele+ABC[no_of_periods+proj_yr]+str(idx)
                    else:
                        if "Avg " in row[keys[key]]:
                            cellref = "average("+ele+colprev+str(idx)+":"+ele+col+str(idx)+")"
                        elif "Delta " in row[keys[key]]:
                            cellref = "("+ele+col+str(idx)+"-"+ele+colprev+str(idx)+")"
                        else:
                            cellref = ele+col+str(idx)
                            print(cellref)
                        #If a cell has not been calculated before or no assumption, then assume prev yr value
                        #if sheetref.range(cellref).value == None:
                        #    sheetref.range(cellref).value = sheetref.range(ele+ABC[no_of_periods+proj_yr]+str(idx)).value
                    #print(element, cellref)#, sheetref.range(cellref).value)

                formula += cellref
            elif "Sign" in keys[key]:
                formula += row[keys[key]] 
            else:
                print("Error: unknown type")
        sheet.range(col+str(l+2)).value = formula
        #print(formula)
        
    if row["Sheet"] == "PL":
        plheads.append(row["Head"])
    elif row["Sheet"] == "BS":
        bsheads.append(row["Head"])
    elif row["Sheet"] == "Ratios":
        ratioheads.append(row["Head"])
    elif row["Sheet"] == "CF":
        cfheads.append(row["Head"])
    else:
        print("Unknown sheet: ", row["Sheet"])

Working on: Gross profit
<Sheet [Book5]PL> A16
PL!C11
PL!C2
PL!D11
PL!D2
PL!E11
PL!E2
PL!F11
PL!F2
Working on: EBITDA
<Sheet [Book5]PL> A17
PL!C16
PL!C4
PL!C12
PL!C9
PL!D16
PL!D4
PL!D12
PL!D9
PL!E16
PL!E4
PL!E12
PL!E9
PL!F16
PL!F4
PL!F12
PL!F9
Working on: PBT
<Sheet [Book5]PL> A18
PL!C17
PL!C3
PL!C7
PL!C10
PL!C5
PL!D17
PL!D3
PL!D7
PL!D10
PL!D5
PL!E17
PL!E3
PL!E7
PL!E10
PL!E5
PL!F17
PL!F3
PL!F7
PL!F10
PL!F5
Working on: PAT
<Sheet [Book5]PL> A19
PL!C18
PL!C14
PL!D18
PL!D14
PL!E18
PL!E14
PL!F18
PL!F14
Working on: Consol PAT
<Sheet [Book5]PL> A20
PL!C19
PL!C8
PL!C13
PL!C6
PL!D19
PL!D8
PL!D13
PL!D6
PL!E19
PL!E8
PL!E13
PL!E6
PL!F19
PL!F8
PL!F13
PL!F6
Working on: Networth
<Sheet [Book5]BS> A20
BS!C7
BS!C16
BS!D7
BS!D16
BS!E7
BS!E16
BS!F7
BS!F16
Working on: Net block
<Sheet [Book5]BS> A21
BS!C8
BS!C2
BS!D8
BS!D2
BS!E8
BS!E2
BS!F8
BS!F2
Working on: Total liabilities
<Sheet [Book5]BS> A22
BS!C20
BS!C18
BS!C5
BS!C13
BS!C15
BS!C11
BS!D20
BS!D18
BS!D5
BS!D13
BS!D15
BS!D11
BS!E20
BS!E18
BS!E5
BS!E13

In [13]:
assump = pd.read_excel(path, sheet_name="Assumptions")
assump.set_index("Head", inplace=True)
sheetassm = book.sheets.add("Assumptions")
sheetassm.range("A1").value=assump
sheetassm.range("D2:E19").color = (150,255,255)

In [14]:
assumps = assump.reset_index().to_dict("records")
fys=["FY23","FY24"]
for proj_yr, fy in enumerate(fys):
    col = ABC[proj_yr+3]
    for i, row in enumerate(assumps):
        variable = row["Head"]
        print(variable)
        #print(row)
        if variable in bsheads:
            ele1 = "BS!"
            idx1 = bsheads.index(variable)+2 #+1 as xl starts from 1 and +1 for header row
            sheet = sheetbs
        elif variable in plheads:
            ele1 = "PL!"
            idx1 = plheads.index(variable)+2
            sheet = sheetpl
        elif variable in ratioheads:
            ele1 = "Ratios!"
            idx1 = ratioheads.index(variable)+2
            sheet = shtratios
        elif variable in cfheads:
            ele1 = "CF!"
            idx1 = cfheads.index(variable)+2
            sheet = sheetcf
        else:
            print("Unknown: ", variable)
            continue
        if row["Method"] == "Assume":
            prev_cellref = ele1+ABC[no_of_periods+proj_yr]+str(idx1)
            curr_cellref = ele1+ABC[no_of_periods+proj_yr+1]+str(idx1)
            print(prev_cellref, curr_cellref)
            if row["Type"] == "Growth":
                sheet.range(curr_cellref).value = "="+prev_cellref+"*(1+"+"Assumptions!"+col+str(i+2)+")"#str(1+row[fy])
            elif row["Type"] == "Add":
                sheet.range(curr_cellref).value = "="+prev_cellref+"+"+"Assumptions!"+col+str(i+2)#str(row[fy])
            elif row["Type"] == "Abs":
                sheet.range(curr_cellref).value = "="+"Assumptions!"+col+str(i+2)#str(row[fy])
            else:
                print("Error: unknown type")
        else:
            print(row)
            print("Error: unknown method")

Sales
PL!F11 PL!G11
GPM
Ratios!F2 Ratios!G2
Employee cost %
Ratios!F4 Ratios!G4
Sales and Mkt %
Ratios!F5 Ratios!G5
Other expenses %
Ratios!F6 Ratios!G6
Depreciation %
Ratios!F7 Ratios!G7
Interest %
Ratios!F8 Ratios!G8
Other income %
Ratios!F9 Ratios!G9
Exceptional expenses
PL!F5 PL!G5
Extraordinary items
PL!F6 PL!G6
Tax rate
Ratios!F10 Ratios!G10
Payout ratio
Ratios!F20 Ratios!G20
Minority interest PL
PL!F8 PL!G8
Share of Assoc
PL!F13 PL!G13
Equity raise
Unknown:  Equity raise
Total debt
BS!F18 BS!G18
Creditor days
Ratios!F17 Ratios!G17
Capex
Unknown:  Capex
CWIP
BS!F3 BS!G3
Long term investments
BS!F10 BS!G10
Other long term assets
BS!F14 BS!G14
Inventory days
Ratios!F18 Ratios!G18
Debtor days
Ratios!F19 Ratios!G19
Short term investments
BS!F17 BS!G17
Other CA %
Ratios!F14 Ratios!G14
Sales
PL!G11 PL!H11
GPM
Ratios!G2 Ratios!H2
Employee cost %
Ratios!G4 Ratios!H4
Sales and Mkt %
Ratios!G5 Ratios!H5
Other expenses %
Ratios!G6 Ratios!H6
Depreciation %
Ratios!G7 Ratios!H7
Interest %
Rati

In [15]:
assump_df = assump.reset_index()
assumpheads = assump_df["Head"].tolist()
formulae = pd.read_excel(path, sheet_name="Iteration", usecols="A:O").to_dict("records")
for proj_yr, fy in enumerate(fys):
    for i, row in enumerate(formulae):
        variable = row["Head"]
        print(variable)

        if variable in bsheads:
            ele1 = "BS!"
            idx1 = bsheads.index(variable)+2 #+1 as xl starts from 1 and +1 for header row
            sheet = sheetbs
        elif variable in plheads:
            ele1 = "PL!"
            idx1 = plheads.index(variable)+2
            sheet = sheetpl
        elif variable in ratioheads:
            ele1 = "Ratios!"
            idx1 = ratioheads.index(variable)+2
            sheet = shtratios
        elif variable in cfheads:
            ele1 = "CF!"
            idx1 = cfheads.index(variable)+2
            sheet = sheetcf
        else:
            print("Unknown: ", variable)
            continue
        curr_cellref = ele1+ABC[no_of_periods+proj_yr+1]+str(idx1)

        keys = list(row.keys())
        formula = "="
        for key in range(1,len(keys)-1): #skipping first and last columns
            if pd.isna(row[keys[key]]):
                continue
            else:
                if "Element" in keys[key]:
                    if isinstance(row[keys[key]], int) or isinstance(row[keys[key]], float):
                        cellref=str(row[keys[key]])
                    else:
                        flag = False
                        if "Avg " in row[keys[key]]:
                            element = row[keys[key]].lstrip("Avg ")
                        elif "Delta " in row[keys[key]]:
                            element = row[keys[key]].lstrip("Delta").lstrip(" ")
                            #print("kkk", row[keys[key]], element)
                        else:
                            element = row[keys[key]]
                        if element in bsheads:
                            #print("BS", element)
                            ele = "BS!"
                            idx = bsheads.index(element)+2 #+1 as xl starts from 1 and +1 for header row
                            sheetref = sheetbs
                        elif element in plheads:
                            ele = "PL!"
                            idx = plheads.index(element)+2
                            sheetref = sheetpl
                        elif element in ratioheads:
                            ele = "Ratios!"
                            idx = ratioheads.index(element)+2
                            sheetref = shtratios
                        elif element in cfheads:
                            ele = "CF!"
                            idx = cfheads.index(element)+2
                            sheetref = sheetcf
                        else:
                            print("Unknown ele: ", element)
                            flag = True

                        if element == variable: #Refers to itself
                            cellref = ele+ABC[no_of_periods+proj_yr]+str(idx)
                        else:
                            if "Avg " in row[keys[key]]:
                                cellref = "average("+ele+ABC[no_of_periods+proj_yr]+str(idx)+":"+ele+ABC[no_of_periods+proj_yr+1]+str(idx)+")"
                            elif "Delta " in row[keys[key]]:
                                cellref = "("+ele+ABC[no_of_periods+proj_yr+1]+str(idx)+"-"+ele+ABC[no_of_periods+proj_yr]+str(idx)+")"
                            else:
                                if flag:
                                    #Check if defined in Assumptions sheet
                                    if element in assumpheads:
                                        cellref = "Assumptions!"+ABC[proj_yr+3]+str(assumpheads.index(element)+2)
                                    else:
                                        cellref=str(0)
                                else:
                                    cellref = ele+ABC[no_of_periods+proj_yr+1]+str(idx)
                                    #If a cell has not been calculated before or no assumption, then assume prev yr value
                                    if sheetref.range(cellref).value == None:
                                        sheetref.range(cellref).value = sheetref.range(ele+ABC[no_of_periods+proj_yr]+str(idx)).value
                        print(element, cellref)#, sheetref.range(cellref).value)
                    formula += cellref
                elif "Sign" in keys[key]:
                    formula += row[keys[key]] 
                else:
                    print("Error: unknown type")
        sheet.range(curr_cellref).value = formula
        #time.sleep(0.5)
# if cash is negative, means debt to increase

Gross profit
Sales PL!G11
GPM Ratios!G2
Employee expenses
Sales PL!G11
Employee cost % Ratios!G4
Sales and Mkt expenses
Sales PL!G11
Sales and Mkt % Ratios!G5
Other expenses
Sales PL!G11
Other expenses % Ratios!G6
EBITDA
Gross profit PL!G16
Employee expenses PL!G4
Sales and Mkt expenses PL!G12
Other expenses PL!G9
Depreciation
Sales PL!G11
Depreciation % Ratios!G7
Interest cost
Total debt average(BS!F18:BS!G18)
Interest % Ratios!G8
Other income
Sales PL!G11
Other income % Ratios!G9
PBT
EBITDA PL!G17
Depreciation PL!G3
Interest cost PL!G7
Other income PL!G10
Exceptional expenses PL!G5
Tax
PBT PL!G18
Tax rate Ratios!G10
PAT
PBT PL!G18
Tax PL!G14
Consol PAT
PAT PL!G19
Minority interest PL PL!G8
Share of Assoc PL!G13
Extraordinary items PL!G6
Dividend
PAT PL!G19
Payout ratio Ratios!G20
Equity share capital
Equity share capital BS!F7
Reserves
Reserves BS!F16
PAT PL!G19
Dividend PL!G15
Networth
Equity share capital BS!G7
Reserves BS!G16
Creditors
Sales PL!G11
Creditor days Ratios!G17
Other c

In [104]:
# Check if cash is negative, increase debt
idx_cash = bsheads.index("Cash and equivalents")+2
idx_debt = assumpheads.index("Total debt")+2
idx_netcash = cfheads.index("Net")+2

for proj_yr, fy in enumerate(fys):
    cashcellref = "BS!"+ABC[no_of_periods+proj_yr+1]+str(idx_cash)
    debtcellref = "Assumptions!"+ABC[proj_yr+3]+str(idx_debt)
    if sheetbs.range(cashcellref).value > 0:
        continue
    else:
        sheetbs.range(cashcellref).api.GoalSeek(100, sheetassm.range(debtcellref).api)


#     #intialize debt value as negative value for net cash movement in that year
#     debt_value = -sheetcf.range("CF!"+ABC[no_of_periods+proj_yr+1]+str(idx_netcash)).value
#     #keep adding 100 mn to debt till cash is >0
#     while sheetbs.range(cashcellref).value <= 0:
#         sheetassm.range(debtcellref).value += debt_value
#         debt_value += 100.0
#         print(sheetbs.range(cashcellref).value)

#debtcellref = "Assumptions!"+ABC[]+str(idx_debt)
#print(sheetassm.range(debtcellref).value)